In [116]:
import os
import re
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)

In [117]:
edge_weights = pd.read_csv("edge_weights_mmhc.txt")  # 'edge_weights_hc.csv'
edge_weights

,from,to,strength
0,I12,N18,-18019.568573
1,E11,Z79_4,-9306.451680
2,F41,F41_1,-9257.156766
3,I13,N18,-7058.647628
4,I25,Z95,-5809.497942
5,I13,I50,-5542.542680
6,ethncty,race,-5407.190724
7,I11,I50,-4900.014944
8,I48,Z79_01,-4127.012369
9,I25,E78_5,-3965.850200


# Export interactive graph to Vis_js

In [118]:
def get_nodes_and_edges_for_strength_table(arc_strength_df, **node_attribute_dicts):
    """
    format arc strength dataframe into separate node and edge dataframes.
    
    Args:
    * arc_strength_df: an edge table as generated by `bn.strength`, where strengths are edge weights
    * node_attribute_dicts: a dictionary of dictionaries where the keys are attribute names 
        (like 'label', 'group', and 'title') and the values are dictionaries mapping nodes to attribute values.
    """

    weight_col='strength'
    all_nodes = [w for w in sorted({v for v in arc_strength_df['from']} | {v for v in arc_strength_df['to']})]

    nodes_df = pd.DataFrame([{'key': node} for node in all_nodes])
    for attrib, attrib_dict in node_attribute_dicts.items():
        nodes_df[attrib] = [attrib_dict.get(str(node).replace('_', '.'), node) for node in all_nodes]

    nodes_df['id'] = nodes_df.index

    node_id = {r['key']:r['id'] for idx, r in nodes_df.iterrows()}

    edges_df = arc_strength_df
    edges_df[weight_col] = -1 * edges_df[weight_col]
    edges_df['from'] = [node_id[nn] for nn in edges_df['from']]
    edges_df['to'] = [node_id[nn] for nn in edges_df['to']]
    
    print("Your graph will have {0} nodes and {1} edges.".format( len(nodes_df), len(edges_df) ))
    
    return nodes_df, edges_df


In [119]:
arc_strength_df = edge_weights

all_nodes = [w for w in sorted({v for v in arc_strength_df['from']} | {v for v in arc_strength_df['to']})]

nodes_df = pd.DataFrame([{'key': node} for node in all_nodes])

arc_strength_df

,from,to,strength
0,I12,N18,-18019.568573
1,E11,Z79_4,-9306.451680
2,F41,F41_1,-9257.156766
3,I13,N18,-7058.647628
4,I25,Z95,-5809.497942
5,I13,I50,-5542.542680
6,ethncty,race,-5407.190724
7,I11,I50,-4900.014944
8,I48,Z79_01,-4127.012369
9,I25,E78_5,-3965.850200


In [120]:
def export_to_vis_js(nodes_df, edges_df, title, html_file_name):
    weight_col = 'strength'
    max_threshold = edges_df[weight_col].quantile(q=0.95) # percentile
    nodes_str = nodes_df.to_json(orient='records')
    edges_str = edges_df.to_json(orient='records')
    
    html_string = ( 
        '<!DOCTYPE html>\n'
        '<html lang="en">\n'
        '<head>\n'
        '	<meta http-equiv="content-type" content="text/html; charset=utf-8" />\n'
        f'	<title>{title}</title>\n'
        '	<script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>\n'
        f'	<script type="text/javascript">NODE_LIST={nodes_str};EDGE_LIST = {edges_str};</script>\n'
        '	<style type="text/css">#mynetwork {width: 100%; height: 700px; border: 1px}</style>\n'
        '	</head>\n'
        '		<body>\n'
        '			<div class="slidercontainer">\n'
        '				<label>minimum edge strength:\n'
        f'					<input type="range" min="0" max="{max_threshold}" value="0" step="1" class="slider" id="min_edge_weight" \n'
        '							onchange="document.getElementById(\'min_edge_weight_display\').value=this.value;">\n'
        '					<input type="text" id="min_edge_weight_display" size="2" value="0.5">\n'
        '				</label>\n'
        '			</div>\n'
        '			<div id="mynetwork"></div>\n'
        '			<script type="text/javascript">\n'
        f'	const edge_weight_metric = "{weight_col}"\n'
        '	for (var i = 0; i < EDGE_LIST.length; i++) {\n'
        '		EDGE_LIST[i]["arrows"] = "to"\n'
        '		EDGE_LIST[i]["value"] = EDGE_LIST[i][edge_weight_metric]\n'
        '	}\n'
        '	\n'
        '	const edgeFilterSlider = document.getElementById("min_edge_weight")\n'
        '	\n'
        '	function edgesFilter(edge){return edge.value > edgeFilterSlider.value}\n'
        '	\n'
        '	const nodes = new vis.DataSet(NODE_LIST)\n'
        '	const edges = new vis.DataSet(EDGE_LIST)\n'
        '	\n'
        '	const nodesView = new vis.DataView(nodes)\n'
        '	const edgesView = new vis.DataView(edges, { filter: edgesFilter })\n'
        '	\n'
        '	edgeFilterSlider.addEventListener("change", (e) => {edgesView.refresh()})\n'
        '	\n'
        '	const container = document.getElementById("mynetwork")\n'
        '	const options = {physics:{maxVelocity: 10, minVelocity: 0.5}}\n'
        '	const data = { nodes: nodesView, edges: edgesView }\n'
        '	new vis.Network(container, data, options)\n'
        '	\n'
        '			</script>\n'
        '		</body>\n'
        '	</html>\n'
    )
    with open(html_file_name, "wt") as html_file:
        html_file.write(html_string)


In [121]:
node_names_pdf = pd.read_csv("C:/Users/rhorton/OneDrive - Microsoft/ACE Team/Mercy/comorbidities/ICD10_taxonomy.txt",
                            delimiter='\t', names=['code', 'label'])

node_label_dict = {r.code:r.code + ': ' + r.label for r in node_names_pdf.itertuples()}

In [122]:
bn_nodes, bn_edges = get_nodes_and_edges_for_strength_table(edge_weights, label=node_label_dict) # group, title


Your graph will have 235 nodes and 331 edges.


In [123]:
export_to_vis_js(bn_nodes, bn_edges, title="selected_nodes", html_file_name="selected_nodes.html")